# Pruebas de Endpoints de la Aplicación E-Commerce

Este Notebook contiene ejemplos de llamadas a los endpoints definidos en la aplicación, abarcando los siguientes casos de uso:

1. **Autenticación y Sesión de Usuarios:** Logueo y consulta de sesión (almacenada en Redis).
2. **Categorización de Usuarios:** Registro y listado de usuarios con su categoría (TOP, MEDIUM, LOW).
3. **Catálogo de Productos:** Crear y listar productos.
4. **Gestión de Carritos de Compras:** Agregar producto al carrito, ver el carrito y confirmar el pedido.
5. **CRUD de Usuarios:** Crear, obtener, actualizar y eliminar un usuario.
6. **CRUD de Productos:** Crear, obtener, actualizar y eliminar un producto.
7. **CRUD de Pedidos:** Crear, obtener, actualizar y eliminar un pedido.

Para mantener la sesión entre solicitudes, se utiliza un objeto `requests.Session()` que conserva las cookies asignadas por Flask.

In [1]:
import requests

# Crear una sesión persistente para mantener la cookie de sesión
session = requests.Session()

# URL base de la aplicación
base_url = "http://localhost:5000"
print("Base URL:", base_url)

Base URL: http://localhost:5000


## 1. Autenticación y Sesión de Usuarios

### 1.1. Login (POST /login)
Se envía el email del usuario (por ejemplo, `alice@example.com`) para que el backend asigne un `session_id` y almacene la información en Redis.

In [2]:
# Endpoint de login
login_url = f"{base_url}/login"

# Datos de login
login_data = {"email": "alice@example.com"}

response_login = session.post(login_url, data=login_data)
print("Login Response:")
print(response_login.status_code)
print(response_login.text)

Login Response:
200
{
  "message": "Usuario logueado",
  "user": "alice@example.com"
}



## 2. Consulta de Sesión en Redis

Este endpoint (`GET /ver_sesion`) debe devolver la información de sesión almacenada en Redis. Esto permite verificar que la sesión se creó correctamente en el login.

In [3]:
# Endpoint para consultar la sesión en Redis
ver_sesion_url = f"{base_url}/ver_sesion"
response_ver_sesion = session.get(ver_sesion_url)

print("Ver Sesión Response:")
print(response_ver_sesion.status_code)
try:
    print(response_ver_sesion.json())
except Exception as e:
    print(response_ver_sesion.text)

Ver Sesión Response:
200
{'user_email': 'alice@example.com', 'user_id': '67b61009891bdeaa8ba8166c'}


## 3. Catálogo de Productos

### 3.1. Crear Producto (POST /productos)
Se envía un JSON con la información del producto.

In [4]:
productos_url = f"{base_url}/productos"

producto_payload = {
    "nombre": "Smartphone X",
    "descripcion": "Gama alta",
    "precio": 999.99,
    "stock": 5
}

response_producto = session.post(productos_url, json=producto_payload)
print("Crear Producto Response:")
print(response_producto.status_code)
print(response_producto.json())

# Guardar el ID del producto creado para usar en pruebas posteriores
product_id = response_producto.json().get('_id')
print("Product ID:", product_id)

Crear Producto Response:
201
{'_id': '67b61061258d596af020a8bd'}
Product ID: 67b61061258d596af020a8bd


### 3.2. Listar Productos (GET /productos)

In [5]:
response_listar_productos = session.get(productos_url)
print("Listado de Productos:")
print(response_listar_productos.status_code)
print(response_listar_productos.json())

Listado de Productos:
200
[{'_id': '67b6100a891bdeaa8ba8166e', 'descripcion': 'Laptop muy potente', 'nombre': 'Laptop Gamer', 'precio': 1500.99, 'stock': 10}, {'_id': '67b6100a891bdeaa8ba8166f', 'descripcion': 'Gama alta', 'nombre': 'Smartphone X', 'precio': 999.99, 'stock': 5}, {'_id': '67b61061258d596af020a8bd', 'descripcion': 'Gama alta', 'nombre': 'Smartphone X', 'precio': 999.99, 'stock': 5}]


## 4. Gestión de Carritos de Compras

### 4.1. Agregar Producto al Carrito (POST /agregar_carrito)
Envía el `product_id` y `cantidad` en una solicitud de formulario. Se asume que la sesión ya está iniciada.

Nota: Se valida que el producto exista en MongoDB antes de agregarlo.

In [6]:
# Endpoint para agregar producto al carrito
agregar_carrito_url = f"{base_url}/agregar_carrito"

# Datos del carrito
carrito_data = {
    "product_id": product_id,  # Usamos el ID del producto creado anteriormente
    "cantidad": "2"
}

response_agregar = session.post(agregar_carrito_url, data=carrito_data)
print("Agregar Carrito Response:")
print(response_agregar.status_code)
print(response_agregar.json())

Agregar Carrito Response:
200
{'message': 'Producto agregado al carrito'}


## 5. Ver Carrito (GET /ver_carrito)

Se consulta el contenido del carrito asociado a la sesión actual.

In [9]:
# Endpoint para ver el carrito
ver_carrito_url = f"{base_url}/ver_carrito"
response_ver_carrito = session.get(ver_carrito_url)
print("Ver Carrito Response:")
print(response_ver_carrito.status_code)
print(response_ver_carrito.json())

Ver Carrito Response:
200
{}


## 6. Confirmar Pedido (POST /confirmar_pedido)

Este endpoint debe leer el carrito desde Redis, crear un pedido en MongoDB y limpiar el carrito.

In [8]:
# Endpoint para confirmar el pedido
confirmar_pedido_url = f"{base_url}/confirmar_pedido"
response_confirmar = session.post(confirmar_pedido_url)
print("Confirmar Pedido Response:")
print(response_confirmar.status_code)
print(response_confirmar.json())

Confirmar Pedido Response:
200
{'message': 'Pedido confirmado', 'total': 1999.98}


## 7. CRUD de Usuarios

In [ ]:
# Crear un usuario para pruebas CRUD
user_payload = {
    "nombre": "Test User",
    "email": "testuser@example.com",
    "categoria": "LOW"
}
response_create_user = session.post(usuarios_url, json=user_payload)
print('Crear Usuario Response:')
print(response_create_user.status_code)
print(response_create_user.json())

# Guardamos el ID del usuario creado para usar en pruebas posteriores
user_id = response_create_user.json().get('_id')
print('User ID:', user_id)

In [ ]:
# Obtener el usuario creado
get_user_url = f"{usuarios_url}/{user_id}"
response_get_user = session.get(get_user_url)
print('Get Usuario Response:')
print(response_get_user.status_code)
print(response_get_user.json())

In [ ]:
# Actualizar el usuario creado
update_payload = {
    "nombre": "Test User Updated",
    "categoria": "MEDIUM"
}
update_user_url = f"{usuarios_url}/{user_id}"
response_update_user = session.put(update_user_url, json=update_payload)
print('Update Usuario Response:')
print(response_update_user.status_code)
print(response_update_user.json())

In [ ]:
# Eliminar el usuario creado
delete_user_url = f"{usuarios_url}/{user_id}"
response_delete_user = session.delete(delete_user_url)
print('Delete Usuario Response:')
print(response_delete_user.status_code)
print(response_delete_user.json())

## 8. CRUD de Productos

In [ ]:
# Crear un producto para pruebas CRUD
product_payload = {
    "nombre": "Test Product",
    "descripcion": "Producto de prueba",
    "precio": 123.45,
    "stock": 50
}
response_create_product = session.post(productos_url, json=product_payload)
print('Crear Producto Response:')
print(response_create_product.status_code)
print(response_create_product.json())

# Guardamos el ID del producto creado para usar en pruebas posteriores
product_id = response_create_product.json().get('_id')
print('Product ID:', product_id)

In [ ]:
# Obtener el producto creado
get_product_url = f"{productos_url}/{product_id}"
response_get_product = session.get(get_product_url)
print('Get Producto Response:')
print(response_get_product.status_code)
print(response_get_product.json())

In [ ]:
# Actualizar el producto creado
update_product_payload = {
    "precio": 150.00,
    "stock": 45
}
update_product_url = f"{productos_url}/{product_id}"
response_update_product = session.put(update_product_url, json=update_product_payload)
print('Update Producto Response:')
print(response_update_product.status_code)
print(response_update_product.json())

In [ ]:
# Eliminar el producto creado
delete_product_url = f"{productos_url}/{product_id}"
response_delete_product = session.delete(delete_product_url)
print('Delete Producto Response:')
print(response_delete_product.status_code)
print(response_delete_product.json())

## 9. CRUD de Pedidos

Esta sección prueba la creación, consulta, actualización y eliminación de un pedido.

Nota: Para crear un pedido, se requiere enviar un `usuario_id` (que debe ser válido) y un arreglo de items (cada uno con `product_id`, `cantidad`, `precio_unitario` y `subtotal`).

In [ ]:
# Crear un pedido para pruebas CRUD
pedido_payload = {
    "usuario_id": "SOME_VALID_USER_ID",  # Reemplaza con un ID de usuario válido
    "items": [
         {"product_id": product_id, "cantidad": 2, "precio_unitario": 100.0, "subtotal": 200.0}
    ],
    "total": 200.0,
    "estado": "pendiente"
}
pedidos_url = f"{base_url}/pedidos"
response_create_pedido = session.post(pedidos_url, json=pedido_payload)
print('Crear Pedido Response:')
print(response_create_pedido.status_code)
print(response_create_pedido.json())

# Guardamos el ID del pedido creado para usar en pruebas posteriores
pedido_id = response_create_pedido.json().get('_id')
print('Pedido ID:', pedido_id)

In [ ]:
# Obtener el pedido creado
get_pedido_url = f"{pedidos_url}/{pedido_id}"
response_get_pedido = session.get(get_pedido_url)
print('Get Pedido Response:')
print(response_get_pedido.status_code)
print(response_get_pedido.json())

In [ ]:
# Actualizar el pedido creado (por ejemplo, cambiar el estado a 'confirmado')
update_pedido_payload = {
    "estado": "confirmado"
}
update_pedido_url = f"{pedidos_url}/{pedido_id}"
response_update_pedido = session.put(update_pedido_url, json=update_pedido_payload)
print('Update Pedido Response:')
print(response_update_pedido.status_code)
print(response_update_pedido.json())

In [ ]:
# Eliminar el pedido creado
delete_pedido_url = f"{pedidos_url}/{pedido_id}"
response_delete_pedido = session.delete(delete_pedido_url)
print('Delete Pedido Response:')
print(response_delete_pedido.status_code)
print(response_delete_pedido.json())